# Coranvirus_Analysis

#### Analysis of Coronavirus (COVID-19) Evolution and Impact

## 1 Import, clean and format confirmed Cases

### 1.1 Data Sources

#### Source of data: https://raw.githubusercontent.com/CSSEGISandData, this is updated by John Hopkins Univiersity from a variety of government and NGO sources
#### For full list of sources see: https://systems.jhu.edu/research/public-health/ncov/
#### originally had been pulling directly from google sheets as per

#### csv_url='http://spreadsheets.google.com/ccc?key=1UF2pSkFTURko2OvfHWWlFpDFAr1UxCBA4JLwlSP6KFo&output=csv'



In [138]:


import pandas as pd
import numpy as np
import requests as rs
import pandas as pd

csv_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'

res=rs.get(url=csv_url)
open('coronavirus_stats_conf2.csv', 'wb').write(res.content)

read_conf = pd.read_csv('/Users/neil.watt/Documents/PythonScripts/Coronavirus/coronavirus_stats_conf2.csv', encoding = "ISO-8859-1")

#read_conf = pd.read_csv('/Users/neilwatt/Documents/Projects/Coronavirus/Coronavirus_Analysis/coronavirus_stats_conf2.csv', encoding = "ISO-8859-1")
#replace blank province/state with nil
read_conf['Province/State'].fillna('', inplace=True)

#replace all other NaNs with 0
read_conf.fillna(0, inplace=True)

#create new name key column
read_conf.insert(1, 'name', read_conf["Province/State"].map(str) +' '+ read_conf["Country/Region"])



### 1.2 Formatting and Cleaning Data

#### The data in the CSV file needs to be re-worked and reshaped to facilitate convenient plotting.
#### 1.  I first sliced the csv into two dataframes and then transposed the date columns (from column 5 onwards)
#### 2. Next step was to rework the cases figures into a "Cases" column according to the Date column (which is created from the index date)

In [139]:


#slice confirmed cases into two dfs in order to recombine transpose of the cases table with the regions

read_conf_copy=read_conf.copy()

#this creates a dataframe which is a slice with just the city rows (which will be replciated for each date)

new_city=read_conf_copy.iloc[:,0:5]
read_conf_copy=read_conf.copy()

#this slices the date columns from the original

#new_df2=read_conf_copy.iloc[:,6:]
new_df2=read_conf_copy.iloc[:,5:]



#transpose date and cases columnes
new_df2_T=new_df2.T


#number cities
i_points=len(read_conf.index)

#new_df2_T_1=new_df2_T.iloc[:,0:1]
new_df2_T['Date'] = new_df2_T.index

d = {}
for i in range (0,i_points):
    d[i]=new_df2_T.iloc[:,i].to_frame()
    d[i].rename(columns={ d[i].columns[0]: "Cases" }, inplace = True)
    d[i]['Date'] = d[i].index
    
   


#### Dataframes are subsequently re-combined and duplicates are eliminated

In [140]:

#recombine dataframes

dfs={}
for i in range (0,i_points):


#match with df1 based on column header
    city_row=new_city.loc[i,:].to_frame()
    city_row_T=city_row.T

#needs to be repeated same number of times as date columns
    n_times=len(d[i].index)
    city_row_T2=pd.concat([city_row_T]*n_times)


    d[i]['tmp'] = 1
    city_row_T2['tmp'] = 1

    dfs[i] = pd.merge(city_row_T2,d[i], on=['tmp'])
    dfs[i] = dfs[i].drop('tmp', axis=1)
    dfs[i]= dfs[i].drop_duplicates(subset='Date')







In [141]:


#append dataframes into master format
from datetime import datetime


final_df = pd.concat(dfs, ignore_index=True)
#in order to model number of days from onset, convert to datetime
final_df['Date_proper'] = pd.to_datetime(final_df['Date'])

#Create new days column based on number of days from first row
final_df['Day']=((final_df['Date_proper']-final_df['Date_proper'].iloc[0]).dt.total_seconds())/(24*60*60)+1


final_df.to_csv(r'Coronavirus_cases_cleaned.csv')

In [142]:

#exlcude Mainland China cases for international cases
final_df_international=final_df[final_df['Country/Region']!='Mainland China']


#mainland china df
final_df_MChina=final_df[final_df['Country/Region']=='Mainland China']

#mainland exlcuding Hubei
final_df_MChina_exHubei=final_df_MChina[final_df_MChina['Province/State']!='Hubei']




## Import, clean and format deaths and recovereries

#### import deaths

In [143]:
#import dead 

dead_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
dead_res=rs.get(url=dead_url)
open('coronavirus_stats_dead.csv', 'wb').write(dead_res.content)




#read_dead = pd.read_csv('/Users/neilwatt/Documents/Projects/Coronavirus/Coronavirus_Analysis/coronavirus_stats_dead.csv', encoding = "ISO-8859-1")
read_dead = pd.read_csv('/Users/neil.watt/Documents/PythonScripts/Coronavirus/coronavirus_stats_dead.csv', encoding = "ISO-8859-1")
#replace blank province/state with nil
read_dead['Province/State'].fillna('', inplace=True)

#replace all other NaNs with 0
read_dead.fillna(0, inplace=True)

#create new name key column
read_dead.insert(1, 'name', read_dead["Province/State"].map(str) +' '+ read_dead["Country/Region"])

#### import recovered

In [144]:
#import recovered

rec_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'
rec_res=rs.get(url=rec_url)
open('coronavirus_stats_rec.csv', 'wb').write(rec_res.content)


#read_rec = pd.read_csv('/Users/neilwatt/Documents/Projects/Coronavirus/Coronavirus_Analysis/coronavirus_stats_rec.csv', encoding = "ISO-8859-1")

read_rec = pd.read_csv('/Users/neil.watt/Documents/PythonScripts/Coronavirus/coronavirus_stats_rec.csv', encoding = "ISO-8859-1")

#replace blank province/state with nil
read_rec['Province/State'].fillna('', inplace=True)

#replace all other NaNs with 0
read_rec.fillna(0, inplace=True)

#create new name key column
read_rec.insert(1, 'name', read_rec["Province/State"].map(str) +' '+ read_rec["Country/Region"])


#### format dead

In [145]:
#slice dead cases into two dfs in order to recombine transpose of the cases table with the regions

read_dead_copy=read_dead.copy()

#this creayes a dataframe which is a slice with just the city rows (which will be replciated for each date)
new_city_dead=read_dead_copy.iloc[:,0:5]
read_dead_copy=read_dead.copy()

#this slices the date columns from the original
new_df2_dead=read_dead_copy.iloc[:,5:]

#transpose date and cases columnes
new_df2_T_dead=new_df2_dead.T


#number cities
i_points_dead=len(read_dead.index)

#new_df2_T_1=new_df2_T.iloc[:,0:1]
new_df2_T_dead['Date'] = new_df2_T_dead.index

d_dead = {}
for i in range (0,i_points):
    d_dead[i]=new_df2_T_dead.iloc[:,i].to_frame()
    d_dead[i].rename(columns={ d_dead[i].columns[0]: "Cases" }, inplace = True)
    d_dead[i]['Date'] = d_dead[i].index
    

In [146]:
#recombine dataframes (deaths)

dfs_dead={}
for i in range (0,i_points_dead):


#match with df1 based on column header
    city_row_dead=new_city_dead.loc[i,:].to_frame()
    city_row_T_dead=city_row_dead.T

#needs to be repeated same number of times as date columns
    n_times_dead=len(d_dead[i].index)
    city_row_T2_dead=pd.concat([city_row_T_dead]*n_times_dead)


    d_dead[i]['tmp'] = 1
    city_row_T2_dead['tmp'] = 1

    dfs_dead[i] = pd.merge(city_row_T2_dead,d_dead[i], on=['tmp'])
    dfs_dead[i] = dfs_dead[i].drop('tmp', axis=1)
    dfs_dead[i]= dfs_dead[i].drop_duplicates(subset='Date')




In [147]:
#append dataframes into master format
from datetime import datetime


final_df_dead = pd.concat(dfs_dead, ignore_index=True)
#in order to model number of days from onset, convert to datetime
final_df_dead['Date_proper'] = pd.to_datetime(final_df_dead['Date'])

#Create new days column based on number of days from first row
final_df_dead['Day']=((final_df_dead['Date_proper']-final_df_dead['Date_proper'].iloc[0]).dt.total_seconds())/(24*60*60)+1


final_df_dead.to_csv(r'Coronavirus_cases_dead_cleaned.csv')

In [148]:
#exlcude Mainland China cases for international cases
final_df_dead_international=final_df_dead[final_df_dead['Country/Region']!='Mainland China']


#mainland china df
final_df_dead_MChina=final_df_dead[final_df_dead['Country/Region']=='Mainland China']

#mainland exlcuding Hubei
final_df_dead_MChina_exHubei=final_df_dead_MChina[final_df_dead_MChina['Province/State']!='Hubei']

#hubei
final_df_dead_MChina_Hubei=final_df_dead_MChina[final_df_dead_MChina['Province/State']=='Hubei']

#### format recoveries

In [149]:
#slice dead cases into two dfs in order to recombine transpose of the cases table with the regions

read_rec_copy=read_rec.copy()

#this creayes a dataframe which is a slice with just the city rows (which will be replciated for each date)
new_city_rec=read_rec_copy.iloc[:,0:5]
read_rec_copy=read_rec.copy()

#this slices the date columns from the original
new_df2_rec=read_rec_copy.iloc[:,5:]

#transpose date and cases columnes
new_df2_T_rec=new_df2_rec.T


#number cities
i_points_rec=len(read_rec.index)

#new_df2_T_1=new_df2_T.iloc[:,0:1]
new_df2_T_rec['Date'] = new_df2_T_rec.index

d_rec = {}
for i in range (0,i_points):
    d_rec[i]=new_df2_T_rec.iloc[:,i].to_frame()
    d_rec[i].rename(columns={ d_rec[i].columns[0]: "Cases" }, inplace = True)
    d_rec[i]['Date'] = d_rec[i].index
    

KeyboardInterrupt: 

In [ ]:
#recombine dataframes (deaths)

dfs_rec={}
for i in range (0,i_points_rec):


#match with df1 based on column header
    city_row_rec=new_city_rec.loc[i,:].to_frame()
    city_row_T_rec=city_row_rec.T

#needs to be repeated same number of times as date columns
    n_times_rec=len(d_rec[i].index)
    city_row_T2_rec=pd.concat([city_row_T_rec]*n_times_rec)


    d_rec[i]['tmp'] = 1
    city_row_T2_rec['tmp'] = 1

    dfs_rec[i] = pd.merge(city_row_T2_rec,d_rec[i], on=['tmp'])
    dfs_rec[i] = dfs_rec[i].drop('tmp', axis=1)
    dfs_rec[i]= dfs_rec[i].drop_duplicates(subset='Date')


In [ ]:
#append dataframes into master format
from datetime import datetime


final_df_rec = pd.concat(dfs_rec, ignore_index=True)
#in order to model number of days from onset, convert to datetime
final_df_rec['Date_proper'] = pd.to_datetime(final_df_rec['Date'])

#Create new days column based on number of days from first row
final_df_rec['Day']=((final_df_rec['Date_proper']-final_df_rec['Date_proper'].iloc[0]).dt.total_seconds())/(24*60*60)+1


final_df_rec.to_csv(r'Coronavirus_cases_rec_cleaned.csv')

In [ ]:
#exlcude Mainland China cases for international cases
final_df_rec_international=final_df_rec[final_df_rec['Country/Region']!='Mainland China']


#mainland china df
final_df_rec_MChina=final_df_rec[final_df_rec['Country/Region']=='Mainland China']

#mainland exlcuding Hubei
final_df_rec_MChina_exHubei=final_df_rec_MChina[final_df_rec_MChina['Province/State']!='Hubei']

#hubei
final_df_rec_MChina_Hubei=final_df_rec_MChina[final_df_rec_MChina['Province/State']=='Hubei']

### Defining Dataframes

In [ ]:

### adding top regions by number of cases for further analysis


#hubei
final_df_MChina_Hubei=final_df_MChina[final_df_MChina['Province/State']=='Hubei']

#Guangdong
final_df_MChina_Guangdong=final_df_MChina[final_df_MChina['Province/State']=='Guangdong']
#Zhejiang
final_df_MChina_Zhejiang=final_df_MChina[final_df_MChina['Province/State']=='Zhejiang']
#Henan
final_df_MChina_Henan=final_df_MChina[final_df_MChina['Province/State']=='Henan']

#Hunan
final_df_MChina_Hunan=final_df_MChina[final_df_MChina['Province/State']=='Hunan']

#Anhui
final_df_MChina_Anhui=final_df_MChina[final_df_MChina['Province/State']=='Anhui']


#Jiangxi
final_df_MChina_Jiangxi=final_df_MChina[final_df_MChina['Province/State']=='Jiangxi']

## Plotting Deaths and Recoveries vs Cases

In [ ]:
final_df_dead_MChina_Hubei['% Dead']=final_df_dead_MChina_Hubei['Cases']/final_df_MChina_Hubei['Cases']*100

final_df_rec_MChina_Hubei['% Rec']=final_df_rec_MChina_Hubei['Cases']/final_df_MChina_Hubei['Cases']*100


#Hubei Deaths vs cases

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=final_df_dead_MChina_Hubei['Date'], y=final_df_dead_MChina_Hubei['% Dead'], name="% Hubei Deaths"),
    secondary_y=False,
)

# Add traces
fig.add_trace(
    go.Scatter(x=final_df_MChina_Hubei['Date'], y=final_df_rec_MChina_Hubei['% Rec'], name="% Hubei Recoveries"),
    secondary_y=False,
)

#fig.add_trace(
 #   go.Scatter(x=final_df_MChina_Hubei['Date'], y=final_df_MChina_Hubei['Cases'], name="Hubei Cases"),
  #  secondary_y=True,
#)

# Add figure title
fig.update_layout(
    title_text="Hubei % Recoveries and Deaths"
)



# Set y-axes titles
fig.update_yaxes(title_text="% Deaths and Recoveries", secondary_y=False)
#fig.update_yaxes(title_text="Cases", secondary_y=True)

fig.show()

In [ ]:
#Hubei Deaths vs cases

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=final_df_dead_MChina_Hubei['Date'], y=final_df_dead_MChina_Hubei['Cases'], name="Hubei Deaths"),
    secondary_y=False,
)

# Add traces
fig.add_trace(
    go.Scatter(x=final_df_MChina_Hubei['Date'], y=final_df_rec_MChina_Hubei['Cases'], name="Hubei Recoveries"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=final_df_MChina_Hubei['Date'], y=final_df_MChina_Hubei['Cases'], name="Hubei Cases"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Hubei Coronavirus Cases vs Recoveries and Deaths"
)



# Set y-axes titles
fig.update_yaxes(title_text="Deaths and Recoveries", secondary_y=False)
fig.update_yaxes(title_text="Cases", secondary_y=True)

fig.show()

In [ ]:
#Hubei Deaths vs cases

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=final_df_dead_MChina_Hubei['Date'], y=final_df_dead_MChina_Hubei['Cases'], name="Hubei Deaths"),
    secondary_y=False,
)

# Add traces
fig.add_trace(
    go.Scatter(x=final_df_MChina_Hubei['Date'], y=final_df_rec_MChina_Hubei['Cases'], name="Hubei Recoveries"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=final_df_MChina_Hubei['Date'], y=final_df_MChina_Hubei['Cases'], name="Hubei Cases"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Hubei Coronavirus Cases vs Recoveries and Deaths"
)



# Set y-axes titles
fig.update_yaxes(title_text="Deaths and Recoveries", secondary_y=False)
fig.update_yaxes(title_text="Cases", secondary_y=True)

fig.show()



In [ ]:
#mainland excluding Hubei
import plotly.express as px


fig = px.line(final_df_dead_MChina_exHubei,title="Mainland China Reported Coronavirus Deaths (excl Hubei)", x="Date", y="Cases", color="name", line_group="name", hover_name="name",render_mode="svg")
fig.for_each_trace(lambda t: t.update(name=t.name.replace("name=", "")))
fig.show()

## Plotting Cases by Geography

In [ ]:
import  plotly.express as px
#https://plot.ly/python-api-reference/generated/plotly.express.scatter_geo.html
scale=0.1




fig = px.scatter_geo(final_df_MChina , lat="Lat",lon="Long",
                    color="Cases",
                    #size="Cases",
                     size_max=70,
                    size='Cases',
                     #size="Cases",
                     title="Coronavirus Cases by Day (Mainland China))",
                     
                   hover_name="name", 
                     #labels="name",
                    color_continuous_scale=px.colors.sequential.RdBu[::-1],
                     # color_continuous_scale=px.colors.sequential.Plasma,
                     #plotly.express.colors.diverging
                    animation_frame="Date"
                    
                    
                    )
fig.show()

In [ ]:
#non Mainland China only

import  plotly.express as px
#https://plot.ly/python-api-reference/generated/plotly.express.scatter_geo.html




fig = px.scatter_geo(final_df_international , lat="Lat",lon="Long",
                    color="Cases",
                    #size="Cases",
                     size_max=40,
                    size='Cases',
                     #size="Cases",
                     title="Coronavirus Cases by Day (exl Mainland CN))",
                     
                   hover_name="name", 
                     #labels="name",
                  #  color_continuous_scale=px.colors.sequential.RdBu[::-1],
                    # color_continuous_scale=px.colors.sequential.RdBu[::-1],
                     # color_continuous_scale=px.colors.sequential.Plasma,
                     #plotly.express.colors.diverging
                    animation_frame="Date"
                   
                    
                    
                    )
fig.show()

In [ ]:

# plot of international cases

import plotly.express as px


fig = px.line(final_df_international,title="International Cases", x="Date", y="Cases", color="name", line_group="name", hover_name="name",render_mode="svg")
fig.for_each_trace(lambda t: t.update(name=t.name.replace("name=", "")))
fig.show()

In [ ]:
final_df_international

In [ ]:

df_italy=final_df_international[final_df_international['Country/Region']=='Italy']
                                
df_italy.head()

In [ ]:
df_top=final_df_international[final_df_international['Country/Region'].isin(['South Korea','Italy','Iran','Japan','France','Germany','Spain'
,'Singapore'
,'Hong Kong'
,'Kuwait'
,'Switzerland'
,'UK'
])]
df_top.head()

      
      

In [ ]:



# plot of international cases- 



import plotly.express as px


fig = px.line(df_top,title="International Cases (top 12)", x="Date", y="Cases", color="name", line_group="name", hover_name="name",render_mode="svg")
fig.for_each_trace(lambda t: t.update(name=t.name.replace("name=", "")))
fig.show()


In [ ]:

import plotly.express as px


fig = px.line(final_df_MChina_exHubei,title="Mainland China Reported Coronavirus Cases (excl Hubei)", x="Date", y="Cases", color="name", line_group="name", hover_name="name",render_mode="svg")
fig.for_each_trace(lambda t: t.update(name=t.name.replace("name=", "")))
fig.show()

In [ ]:



fig = px.line(final_df_MChina_Hubei,title="Hubei Coronavirus Cases", x="Date", y="Cases", color="name", line_group="name", hover_name="name",render_mode="svg")
fig.for_each_trace(lambda t: t.update(name=t.name.replace("name=", "")))
fig.show()

## Distribution of Cases

In [ ]:
#define exponential function

import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
def func(x, a, b):
    return a * np.exp(b * x) 



In [ ]:
#conf range https://lmfit.github.io/lmfit-py/model.html

Cases=final_df_MChina_Hubei['Cases']
Days=final_df_MChina_Hubei['Day']

#empirical data
xdata = Days
y = Cases
ydata = y




popt, pcov = curve_fit(func, xdata, ydata)


fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.plot(xdata, func(xdata, *popt), 'r-',label='Exp Model: a=%5.3f, b=%5.3f' % tuple(popt))
ax.plot(xdata, ydata, 'b-', label='Reported Cases')
plt.xlabel('Days')
plt.ylabel('Cases')
ax.set_title('Hubei Coronavirus Cases by Day')
ax.legend()

In [ ]:
final_df_MChina_Hubei.head()

## Forecasting Future Cases

In [ ]:
 

final_df_MChina_Hubei['y']=final_df_MChina_Hubei['Cases']
final_df_MChina_Hubei['ds']=final_df_MChina_Hubei['Date']


#Guangdong
final_df_MChina_Guangdong['y']=final_df_MChina_Guangdong['Cases']
final_df_MChina_Guangdong['ds']=final_df_MChina_Guangdong['Date']

#Zhejiang
final_df_MChina_Zhejiang['y']=final_df_MChina_Zhejiang['Cases']
final_df_MChina_Zhejiang['ds']=final_df_MChina_Zhejiang['Date']

#Henan
final_df_MChina_Henan['y']=final_df_MChina_Henan['Cases']
final_df_MChina_Henan['ds']=final_df_MChina_Henan['Date']

#Hunan
final_df_MChina_Hunan['y']=final_df_MChina_Hunan['Cases']
final_df_MChina_Hunan['ds']=final_df_MChina_Hunan['Date']

#Anhui
final_df_MChina_Anhui['y']=final_df_MChina_Anhui['Cases']
final_df_MChina_Anhui['ds']=final_df_MChina_Anhui['Date']

#Jiangxi
final_df_MChina_Jiangxi['y']=final_df_MChina_Jiangxi['Cases']
final_df_MChina_Jiangxi['ds']=final_df_MChina_Jiangxi['Date']




In [ ]:

from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot


#NOTE, DEFAULT changepoint_prior_scale IS 0.05, DECREASING makes less flexible, INCREASIGN makes more flexible
changepoint_prior_scale_new=0.065
periods_new=30


df_Hubei=final_df_MChina_Hubei
gm_prophet = Prophet(interval_width=0.95, changepoint_prior_scale=changepoint_prior_scale_new)
gm_prophet.fit(df_Hubei)
future = gm_prophet.make_future_dataframe(periods=periods_new)



#Guangdong

df_Guangdong=final_df_MChina_Guangdong
gm_prophet_Guangdong = Prophet(interval_width=0.95, changepoint_prior_scale=changepoint_prior_scale_new)
gm_prophet_Guangdong.fit(df_Guangdong)
future_Guangdong = gm_prophet_Guangdong.make_future_dataframe(periods=periods_new)


#Zhejiang
df_Zhejiang=final_df_MChina_Zhejiang
gm_prophet_Zhejiang = Prophet(interval_width=0.95, changepoint_prior_scale=changepoint_prior_scale_new)
gm_prophet_Zhejiang.fit(df_Zhejiang)
future_Zhejiang = gm_prophet_Zhejiang.make_future_dataframe(periods=periods_new)


#Henan
df_Henan=final_df_MChina_Henan
gm_prophet_Henan = Prophet(interval_width=0.95, changepoint_prior_scale=changepoint_prior_scale_new)
gm_prophet_Henan.fit(df_Henan)
future_Henan = gm_prophet_Henan.make_future_dataframe(periods=periods_new)



#Hunan
df_Hunan=final_df_MChina_Hunan
gm_prophet_Hunan = Prophet(interval_width=0.95, changepoint_prior_scale=changepoint_prior_scale_new)
gm_prophet_Hunan.fit(df_Hunan)
future_Hunan = gm_prophet_Hunan.make_future_dataframe(periods=periods_new)


#Anhui
df_Anhui=final_df_MChina_Anhui
gm_prophet_Anhui = Prophet(interval_width=0.95, changepoint_prior_scale=changepoint_prior_scale_new)
gm_prophet_Anhui.fit(df_Anhui)
future_Anhui = gm_prophet_Anhui.make_future_dataframe(periods=periods_new)


#Jiangxi
df_Jiangxi=final_df_MChina_Jiangxi
gm_prophet_Jiangxi = Prophet(interval_width=0.95, changepoint_prior_scale=changepoint_prior_scale_new)
gm_prophet_Jiangxi.fit(df_Jiangxi)
future_Jiangxi = gm_prophet_Jiangxi.make_future_dataframe(periods=periods_new)





In [ ]:
#create forecast with upper and lower bounds
forecast = gm_prophet.predict(future)
#forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

#Guangdong
forecast_Guangdong = gm_prophet_Guangdong.predict(future_Guangdong)


#Zhejiang
forecast_Zhejiang = gm_prophet_Zhejiang.predict(future_Zhejiang)

#Henan

forecast_Henan = gm_prophet_Henan.predict(future_Henan)

#Hunan
forecast_Hunan = gm_prophet_Hunan.predict(future_Hunan)


#Anhui
forecast_Anhui = gm_prophet_Anhui.predict(future_Anhui)

#Jiangxi
forecast_Jiangxi = gm_prophet_Jiangxi.predict(future_Jiangxi)




In [ ]:






fig_Guangdong = gm_prophet_Guangdong.plot(forecast_Guangdong)
changepoints_Guangdong = add_changepoints_to_plot(fig_Guangdong.gca(), gm_prophet_Guangdong, forecast_Guangdong)


fig_Zhejiang = gm_prophet_Zhejiang.plot(forecast_Zhejiang)
changepoints_Zhejiang = add_changepoints_to_plot(fig_Zhejiang.gca(), gm_prophet_Zhejiang, forecast_Zhejiang)

fig_Henan = gm_prophet_Henan.plot(forecast_Henan)
changepoints_Henan = add_changepoints_to_plot(fig_Henan.gca(), gm_prophet_Henan, forecast_Henan)


fig_Hunan = gm_prophet_Hunan.plot(forecast_Hunan)
changepoints_Hunan = add_changepoints_to_plot(fig_Hunan.gca(), gm_prophet_Hunan, forecast_Hunan)


fig_Anhui = gm_prophet_Anhui.plot(forecast_Anhui)
changepoints_Anhui = add_changepoints_to_plot(fig_Anhui.gca(), gm_prophet_Anhui, forecast_Anhui)


fig_Jiangxi = gm_prophet_Jiangxi.plot(forecast_Jiangxi)
changepoints_Jiangxi = add_changepoints_to_plot(fig_Jiangxi.gca(), gm_prophet_Jiangxi, forecast_Jiangxi)

#### Comparing trajectory of new cases for various provinces (Mainland China)

In [ ]:

# plot major provinces showing similar change in trend (after redfinition of "infected")
#https://www.taiwannews.com.tw/en/news/3874490

fig = plt.figure(figsize=(10, 10))


ax1 = fig.add_subplot(4, 1, 1)
gm_prophet_Anhui.plot(forecast_Anhui, ax=ax1)
ax1.set_title('Anhui, Confirmed Coronavirus Cases')
changepoints = add_changepoints_to_plot(fig.gca(), gm_prophet_Anhui, forecast_Anhui)



ax2 = fig.add_subplot(4, 1, 2)
gm_prophet_Guangdong.plot(forecast_Guangdong, ax=ax2)
changepoints_Guangdong = add_changepoints_to_plot(fig.gca(), gm_prophet_Guangdong, forecast_Guangdong)
ax2.set_title('Guangdong, Confirmed Coronavirus Cases')

ax3 = fig.add_subplot(4, 1, 3)
changepoints_Henan = add_changepoints_to_plot(fig.gca(), gm_prophet_Henan, forecast_Henan)
gm_prophet_Henan.plot(forecast_Henan, ax=ax3)
ax3.set_title('Henan, Confirmed Coronavirus Cases')



ax4 = fig.add_subplot(4, 1, 4)
changepoints_Hunan = add_changepoints_to_plot(fig.gca(), gm_prophet_Hunan, forecast_Hunan)
gm_prophet_Hunan.plot(forecast_Hunan, ax=ax4)
ax4.set_title('Hunan, Confirmed Coronavirus Cases')




axes = fig.get_axes()
axes[0].set_xlabel('')
axes[0].set_ylabel('Cases')
axes[1].set_xlabel('')
axes[1].set_ylabel('Cases')
axes[2].set_xlabel('')
axes[2].set_ylabel('Cases')
axes[3].set_xlabel('Date')
axes[3].set_ylabel('Cases')

fig.savefig('temp.png', dpi=100)


In [ ]:
fig1 = gm_prophet.plot(forecast)
plt.title('Hubei Coronavirus Cases by Day, actual and forecasted');
axes = fig1.get_axes()
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Cases')


In [ ]:
#added change points to forecast
from fbprophet.plot import add_changepoints_to_plot
fig2= gm_prophet.plot(forecast)
a = add_changepoints_to_plot(fig2.gca(), gm_prophet, forecast)
plt.title('Hubei Coronavirus Cases by Day, actual and forecasted with changepoints');
axes = fig2.get_axes()
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Cases')





## Flights Analysis

In [ ]:
#https://pcfg.eu/posts/how-to-plot-flight-routes-using-plotly/

# Import libraries
import pandas as pd
#import plotly.plotly as py
import plotly.offline as ol
from geographiclib.geodesic import Geodesic
geod = Geodesic.WGS84

# Define function to calculate distance (in meters) between two points
def dist(p1Lat, p1Lon, p2Lat, p2Lon):
    return geod.Inverse(p1Lat, p1Lon, p2Lat, p2Lon, Geodesic.DISTANCE)['s12']

# Read the data into a dataframe (specifying the column names)

#read_conf = pd.read_csv('/Users/neil.watt/Documents/PythonScripts/Coronavirus/coronavirus_stats_conf2.csv', encoding = "ISO-8859-1")


#df = pd.read_csv('routes.dat', sep=',', header=None,
 #                names=['Airline','Airline ID','Source airport','Source airport ID',
  #                      'Destination airport','Destination airport ID','Codeshare',
   #                     'Stops','Equipment'])



df = pd.read_csv('/Users/neil.watt/Documents/PythonScripts/Coronavirus/routes.dat.txt',sep=',', header=None,
                 names=['Airline','Airline ID','Source airport','Source airport ID',
                        'Destination airport','Destination airport ID','Codeshare',
                        'Stops','Equipment'])



In [ ]:
# Remove duplicates (only one trajectory per route)
df = df[['Source airport','Destination airport']].drop_duplicates(keep='first', inplace=False)

# Read the data into a dataframe (specifying the column names)
df_airports = pd.read_csv('/Users/neil.watt/Documents/PythonScripts/Coronavirus/airports.dat.txt', sep=',', header=None,
                          names=['Airport','Name','City','Country','IATA','ICAO',
                                 'Latitude','Longitude','Altitude','Timezone','DST',
                                 'Tz','Type','Source'])

In [ ]:
## ALL LONDON AIRPORTS

# Select only those routes starting or ending from a London Airport
# in order London City, Heathrow, Gatwick, Luton, Stansted, Southend
df = df.loc[(df['Source airport'].isin(['LCY','LHR','LGW','LTN','STN','SEN'])) | (df['Destination airport'].isin(['LCY','LHR','LGW','LTN','STN','SEN']))]

# Append the origin airport's coordinates to the routes' dataframe
df = pd.merge(df, df_airports[['IATA','Latitude','Longitude']],
              how='inner', left_on='Source airport', right_on='IATA', suffixes=('_Orig','_Dest'))

# Append the destination airport's coordinates to the routes' dataframe
df = pd.merge(df, df_airports[['IATA','Latitude','Longitude']],
              how='inner', left_on='Destination airport', right_on='IATA', suffixes=('_Orig','_Dest'))

# Keep only Origin/Destination IATA ID columns, and their Latitude/Longitude
df = df.drop(columns=['Source airport','Destination airport'])

# Calculate the distance (great circle distance) between the origin and destination airports
df['Distance'] = ''
for index, row in df.iterrows():
    df.loc[df.index==index,'Distance'] = dist(row.Latitude_Orig, row.Longitude_Orig, row.Latitude_Dest, row.Longitude_Dest)/1000

# Initialise the data list that will be used to feed the plot
data = []

# Append all airports (blue dots) to the map
data.append(dict(
                type = 'scattergeo',
                locationmode = 'ISO-3',
                showlegend = False,
                lon = df_airports['Longitude'],
                lat = df_airports['Latitude'],
                hoverinfo = 'text',
                text = df_airports['IATA'],
                mode = 'markers',
                marker = dict(
                    size=2,
                    color='rgb(0, 0, 255)',
                    line = dict(
                        width=3,
                        color='rgba(68, 68, 68, 0)'
                    )
                ))
        )

# Append the longest route to the map
data.append(
        dict(
            type = 'scattergeo',
            locationmode = 'ISO-3',
            name = 'Longest Route',
            showlegend = True,
            lon = [ df.loc[df['Distance']==df['Distance'].max(),'Longitude_Orig'].values[0], df.loc[df['Distance']==df['Distance'].max(),'Longitude_Dest'].values[0] ],
            lat = [ df.loc[df['Distance']==df['Distance'].max(),'Latitude_Orig'].values[0], df.loc[df['Distance']==df['Distance'].max(),'Latitude_Dest'].values[0] ],
            mode = 'lines',
            line = dict(
                width = 2,
                color = 'red',
            ),
            opacity = 1,
        )
    )

# Append all other routes to the map
for i in range(len(df)):
    data.append(
        dict(
            type = 'scattergeo',
            locationmode = 'ISO-3',
            name = str(df['IATA_Orig'][i]) + ' - ' + str(df['IATA_Dest'][i]),
            showlegend = True if df['IATA_Orig'][i] in ['LCY','LHR','LGW','LTN','STN','SEN'] else False,
            lon = [ df['Longitude_Orig'][i], df['Longitude_Dest'][i] ],
            lat = [ df['Latitude_Orig'][i], df['Latitude_Dest'][i] ],
            mode = 'lines',
            line = dict(
                width = 1,
                color = 'green',
            ),
            opacity = 0.3,
        )
    )

# Define the plot's layout
layout = dict(
        title = 'Airports and Routes',
        showlegend = True,
        geo = dict(
            scope='world',
            projection=dict( type='azimuthal equal area' ),
            showland = True,
            landcolor = 'rgb(255, 255, 255)',
            countrycolor = 'rgb(0, 0, 0)',
        ),
    )

# Create the figure to be plotted
fig = dict( data=data, layout=layout )
#py.plot(fig, world_readable=True)
ol.plot(fig, filename='Airports and routes.html')

In [218]:
## Indian Airports


df_India=df_airports[df_airports['Country']=='India']
India_list=df_India['IATA'].tolist() 
India_list_new=list(set(India_list))


df_Iran=df_airports[df_airports['Country']=='Iran']
Iran_list=df_Iran['IATA'].tolist() 
Iran_list_new=list(set(Iran_list))

df_China=df_airports[df_airports['Country']=='China']
China_list=df_China['IATA'].tolist() 
China_list_new=list(set(China_list))

df_Italy=df_airports[df_airports['Country']=='Italy']
Italy_list=df_Italy['IATA'].tolist() 
Italy_list_new=list(set(Italy_list))

df_UK=df_airports[df_airports['Country']=='United Kingdom']
UK_list=df_UK['IATA'].tolist() 
UK_list_new=list(set(UK_list))

#final_df_MChina_Hubei=final_df_MChina[final_df_MChina['Province/State']=='Hubei']

#inal_df_MChina_Hubei=final_df_MChina[final_df_MChina['Province/State']=='Hubei']



In [219]:
UK_list_new

['EOI',
 'ILY',
 'BZZ',
 'NQT',
 'PPW',
 'BOH',
 'LPL',
 'OBN',
 'ENK',
 'PZE',
 'LSI',
 'GLA',
 'VLY',
 'WTN',
 'SEN',
 'CBG',
 'OUK',
 'SWS',
 'FIE',
 'ESH',
 'HRT',
 'BBP',
 'QFO',
 'QUY',
 'HYC',
 'EDI',
 'QUG',
 'GLO',
 'NHT',
 'BQH',
 'LHR',
 '\\N',
 'KOI',
 'BHD',
 'PLH',
 'DND',
 'SCS',
 'HLE',
 'LMO',
 'LKZ',
 'WIC',
 'RCS',
 'SQZ',
 'UNT',
 'KRH',
 'HAW',
 'MHZ',
 'NCL',
 'BRS',
 'EXT',
 'WRY',
 'SOY',
 'FSS',
 'DSA',
 'OXF',
 'HUY',
 'MAN',
 'LYX',
 'BEX',
 'NDY',
 'PSL',
 'CVT',
 'NQY',
 'WRT',
 'LBA',
 'TRE',
 'NWI',
 'LWK',
 'NRL',
 'CSA',
 'ISC',
 'BLK',
 'BHX',
 'LGW',
 'ADX',
 'FAB',
 'QLA',
 'QCY',
 'ODH',
 'GBA',
 'SOU',
 'MSE',
 'LEQ',
 'YEO',
 'FFD',
 'CAX',
 'EMA',
 'BRR',
 'LDY',
 'LYM',
 'CEG',
 'CAL',
 'BWF',
 'LYE',
 'STN',
 'PIK',
 'LCY',
 'BBS',
 'SYY',
 'ABZ',
 'MME',
 'BEB',
 'LTN',
 'INV',
 'BEQ',
 'FZO',
 'KNF',
 'BFS',
 'CWL']

In [ ]:
['LCY','LHR','LGW','LTN','STN','SEN']

In [220]:
# Import libraries
import pandas as pd
#import plotly.plotly as py
import plotly.offline as ol
from geographiclib.geodesic import Geodesic
geod = Geodesic.WGS84

# Define function to calculate distance (in meters) between two points
def dist(p1Lat, p1Lon, p2Lat, p2Lon):
    return geod.Inverse(p1Lat, p1Lon, p2Lat, p2Lon, Geodesic.DISTANCE)['s12']

# Read the data into a dataframe (specifying the column names)

#read_conf = pd.read_csv('/Users/neil.watt/Documents/PythonScripts/Coronavirus/coronavirus_stats_conf2.csv', encoding = "ISO-8859-1")


#df = pd.read_csv('routes.dat', sep=',', header=None,
 #                names=['Airline','Airline ID','Source airport','Source airport ID',
  #                      'Destination airport','Destination airport ID','Codeshare',
   #                     'Stops','Equipment'])



df = pd.read_csv('/Users/neil.watt/Documents/PythonScripts/Coronavirus/routes.dat.txt',sep=',', header=None,
                 names=['Airline','Airline ID','Source airport','Source airport ID',
                        'Destination airport','Destination airport ID','Codeshare',
                        'Stops','Equipment'])


In [221]:
# Remove duplicates (only one trajectory per route)
df = df[['Source airport','Destination airport']].drop_duplicates(keep='first', inplace=False)

# Read the data into a dataframe (specifying the column names)
df_airports = pd.read_csv('/Users/neil.watt/Documents/PythonScripts/Coronavirus/airports.dat.txt', sep=',', header=None,
                          names=['Airport','Name','City','Country','IATA','ICAO',
                                 'Latitude','Longitude','Altitude','Timezone','DST',
                                 'Tz','Type','Source'])

In [222]:
## ALL LONDON AIRPORTS

# Select only those routes starting or ending from a London Airport
# in order London City, Heathrow, Gatwick, Luton, Stansted, Southend

Country_List=UK_list_new

#India_list_new

# note pipemeans OR
#df = df.loc[(df['Source airport'].isin(Iran_list_new)) | (df['Destination airport'].isin(India_list_new))]
df = df.loc[(df['Source airport'].isin(Country_List)) | (df['Destination airport'].isin(Country_List))]

#df = df.loc[(df['Source airport'].isin(['LCY','LHR','LGW','LTN','STN','SEN'])) | (df['Destination airport'].isin(['LCY','LHR','LGW','LTN','STN','SEN']))]

# Append the origin airport's coordinates to the routes' dataframe
df = pd.merge(df, df_airports[['IATA','Latitude','Longitude']],
              how='inner', left_on='Source airport', right_on='IATA', suffixes=('_Orig','_Dest'))

# Append the destination airport's coordinates to the routes' dataframe
df = pd.merge(df, df_airports[['IATA','Latitude','Longitude']],
              how='inner', left_on='Destination airport', right_on='IATA', suffixes=('_Orig','_Dest'))

# Keep only Origin/Destination IATA ID columns, and their Latitude/Longitude
df = df.drop(columns=['Source airport','Destination airport'])

# Calculate the distance (great circle distance) between the origin and destination airports
df['Distance'] = ''
for index, row in df.iterrows():
    df.loc[df.index==index,'Distance'] = dist(row.Latitude_Orig, row.Longitude_Orig, row.Latitude_Dest, row.Longitude_Dest)/1000

# Initialise the data list that will be used to feed the plot
data = []

# Append all airports (blue dots) to the map
data.append(dict(
                type = 'scattergeo',
                locationmode = 'ISO-3',
                showlegend = False,
                lon = df_airports['Longitude'],
                lat = df_airports['Latitude'],
                hoverinfo = 'text',
                text = df_airports['IATA'],
                mode = 'markers',
                marker = dict(
                    size=2,
                    color='rgb(0, 0, 255)',
                    line = dict(
                        width=3,
                        color='rgba(68, 68, 68, 0)'
                    )
                ))
        )

# Append the longest route to the map
data.append(
        dict(
            type = 'scattergeo',
            locationmode = 'ISO-3',
            name = 'Longest Route',
            showlegend = True,
            lon = [ df.loc[df['Distance']==df['Distance'].max(),'Longitude_Orig'].values[0], df.loc[df['Distance']==df['Distance'].max(),'Longitude_Dest'].values[0] ],
            lat = [ df.loc[df['Distance']==df['Distance'].max(),'Latitude_Orig'].values[0], df.loc[df['Distance']==df['Distance'].max(),'Latitude_Dest'].values[0] ],
            mode = 'lines',
            line = dict(
                width = 2,
                color = 'red',
            ),
            opacity = 1,
        )
    )


#India_list_new

# Append all other routes to the map
for i in range(len(df)):
    data.append(
        dict(
            type = 'scattergeo',
            locationmode = 'ISO-3',
            name = str(df['IATA_Orig'][i]) + ' - ' + str(df['IATA_Dest'][i]),
            #showlegend = True if df['IATA_Orig'][i] in India_list_new else False,
            #showlegend = True if df['IATA_Orig'][i] in ['LCY','LHR','LGW','LTN','STN','SEN'] else False,
            lon = [ df['Longitude_Orig'][i], df['Longitude_Dest'][i] ],
            lat = [ df['Latitude_Orig'][i], df['Latitude_Dest'][i] ],
            mode = 'lines',
            line = dict(
                width = 1,
                color = 'green',
            ),
            opacity = 0.3,
        )
    )

# Define the plot's layout
layout = dict(
        title = 'UK Airports and Routes',
      
    title_x=0.5,
    showlegend = False,
        geo = dict(
            scope='world',
            projection=dict( type='azimuthal equal area' ),
            showland = True,
            landcolor = 'rgb(255, 255, 255)',
           # countrycolor = 'rgb(255, 255, 255)',
            showcountries=True, countrycolor="Black"
            #showcountries = True,
            
            
        ),
    #title='Iran Airports and Routes',
    )



# Create the figure to be plotted
fig = dict( data=data, layout=layout )

#py.plot(fig, world_readable=True)
ol.plot(fig, filename='Airports and routes.html')

'Airports and routes.html'

In [5]:
#https://towardsdatascience.com/catching-that-flight-visualizing-social-network-with-networkx-and-basemap-ce4a0d2eaea6

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import mpl_toolkits

from mpl_toolkits.basemap import Basemap as Basemap
#from mpl_toolkits import Basemap as Basemap
#from mpl_toolkits.basemap import Basemap as Basemap

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

In [4]:
graph = nx.from_pandas_dataframe(routes_us, source = 'Source Airport', target = 'Dest Airport', edge_attr = 'number of flights',create_using = nx.DiGraph())

AttributeError: module 'networkx' has no attribute 'from_pandas_dataframe'